### Before start
- Initialize the environment running this command: 
 python -m venv .venv
- Don't forget to add your API key on .env

In [1]:
# Libs to install
!pip install langchain
!pip install python-dotenv
!pip install openai
!pip install pypdf
!pip install bs4
!pip install chromadb -q
!pip install unstructured[local-inference] -q
!pip install python-magic-bin
!pip install selenium
!pip install pydantic-settings
!pip install chromadb
!pip install tiktoken
!pip install fastapi nest-asyncio pyngrok uvicorn


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.4.10 requires pydantic<2.0,>=1.9, but you have pydantic 2.3.0 which is incompatible.
fastapi 0.99.1 requires pydantic!=1.8,!=1.8.1,<2.0.0,>=1.7.4, but you have pydantic 2.3.0 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pydantic-1.10.12-cp311-cp311-win_amd64.whl (2.1 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.3.0
    Uninstalling pydantic-2.3.0:
      Successfully uninstalled pydantic-2.3.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic-settings 2.0.3 requires pydantic>=2.0.1, but you have pydantic 1.10.12 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Libraries & GPT Settings

In [2]:
# Libraries
import os
import openai
import datetime
import uvicorn
import nest_asyncio

from dotenv import load_dotenv, find_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import SeleniumURLLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pyngrok import ngrok


In [3]:
# GPT API settings
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

current_date = datetime.datetime.now().date()

target_date = datetime.date(2024, 6, 12)

if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Functions

In [4]:
def chatWithGPT(prompt, model=llm_model):
    """
    chatWithGPT send the message to ChatGPT API and returns its answer
        :prompt: is the user prompt
        :model: (optional) indicates the GPT model
        :return: returns the answer from ChatGPT
    """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

def getAllData(data_dirpath):
    """
    getAllData loads all data using readFunctions
        :data_dirpath: Directory path of the file that contains all files to load
        :return: returns all data in a string
    """
    # Read PDFs
    pdf_loader = DirectoryLoader(data_dirpath, glob="**/*.pdf")
        
    # Read web URLs in .txt
    with open(data_dirpath + "/" + "webURLs.txt") as f:
        lines = f.readlines()
        f.close()
        
    webpages_loader = SeleniumURLLoader(urls=lines)
    
    loaders = [pdf_loader, webpages_loader]
    documents = []
    
    for loader in loaders:
        documents.extend(loader.load())
                    
    return documents            
    
def getChunkText(documents):
    """
    getChunkText function chunks all text data in chunks
        :text: text data
        :return: chunks of data
    """
    text_splitter = CharacterTextSplitter(
    separator = ".",
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len
    )

    chunks = text_splitter.split_documents(documents)
    return chunks

def chatWithDocs(question, chat_history):
    response = chat_with_docs({"question": question,
                           "chat_history": chat_history})
    return response["answer"]

## Main

### Loading data

In [5]:

documents = getAllData("data")


Error fetching or processing 
, exception: Message: invalid argument
  (Session info: headless chrome=116.0.5845.188)
Stacktrace:
	GetHandleVerifier [0x00007FF68B0252A2+57122]
	(No symbol) [0x00007FF68AF9EA92]
	(No symbol) [0x00007FF68AE6E25D]
	(No symbol) [0x00007FF68AE5C80C]
	(No symbol) [0x00007FF68AE5AA3B]
	(No symbol) [0x00007FF68AE5B153]
	(No symbol) [0x00007FF68AE6FF4B]
	(No symbol) [0x00007FF68AEDEEF7]
	(No symbol) [0x00007FF68AEC6FDA]
	(No symbol) [0x00007FF68AEDEB82]
	(No symbol) [0x00007FF68AEC6DB3]
	(No symbol) [0x00007FF68AE9D2B1]
	(No symbol) [0x00007FF68AE9E494]
	GetHandleVerifier [0x00007FF68B2CEF82+2849794]
	GetHandleVerifier [0x00007FF68B321D24+3189156]
	GetHandleVerifier [0x00007FF68B31ACAF+3160367]
	GetHandleVerifier [0x00007FF68B0B6D06+653702]
	(No symbol) [0x00007FF68AFAA208]
	(No symbol) [0x00007FF68AFA62C4]
	(No symbol) [0x00007FF68AFA63F6]
	(No symbol) [0x00007FF68AF967A3]
	BaseThreadInitThunk [0x00007FFB427F26AD+29]
	RtlUserThreadStart [0x00007FFB4402AA68+40]


#### documents in the output

In [6]:
print(documents)

[Document(page_content='EXERCISE & Women’s Health\n\nAn eBook by\n\nFOREWORD\n\nESME SOAN\n\nFrom puberty to pregnancy, menarche to menopause, exercise is a form of medicine and a hugely important modality to support women’s health. One in two women in Australia are not sufficiently physically active, a statistic that is contributing to the rates of chronic disease burden for women.\n\n‘Women’s Health’ is an umbrella term, used to describe all manner of health conditions and life stages for women, from puberty, pregnancy, postpartum and menopause. Through each of these life stages, exercise has an important therapeutic application for preventative health. Exercise really is a form of medicine – research has shown that it begins even before birth! In utero, exposures to physical activity can even change phenotype expressions for children of exercising mums, showing that exercise in pregnancy has both short and long term positive health effects for mum and child!\n\nExercising right thro

### Splitting

In [7]:
chunks = getChunkText(documents)

Created a chunk of size 2851, which is longer than the specified 1000
Created a chunk of size 1488, which is longer than the specified 1000
Created a chunk of size 2645, which is longer than the specified 1000
Created a chunk of size 2234, which is longer than the specified 1000
Created a chunk of size 4316, which is longer than the specified 1000
Created a chunk of size 1524, which is longer than the specified 1000
Created a chunk of size 2181, which is longer than the specified 1000
Created a chunk of size 1079, which is longer than the specified 1000
Created a chunk of size 1380, which is longer than the specified 1000
Created a chunk of size 1521, which is longer than the specified 1000
Created a chunk of size 2118, which is longer than the specified 1000
Created a chunk of size 2533, which is longer than the specified 1000
Created a chunk of size 1529, which is longer than the specified 1000
Created a chunk of size 2744, which is longer than the specified 1000
Created a chunk of s

#### chunks in ouput

In [8]:
print(chunks[0])

page_content='EXERCISE & Women’s Health\n\nAn eBook by\n\nFOREWORD\n\nESME SOAN\n\nFrom puberty to pregnancy, menarche to menopause, exercise is a form of medicine and a hugely important modality to support women’s health. One in two women in Australia are not sufficiently physically active, a statistic that is contributing to the rates of chronic disease burden for women.\n\n‘Women’s Health’ is an umbrella term, used to describe all manner of health conditions and life stages for women, from puberty, pregnancy, postpartum and menopause. Through each of these life stages, exercise has an important therapeutic application for preventative health' metadata={'source': 'data\\ESSA-Exercise-Womens-Health-eBook.pdf'}


### Embedding

In [9]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(chunks, embeddings)

### Initialize Langchain - Conversation Retrieval Chain

In [10]:
chat_history = []
chat_with_docs = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever())

#### Tests

In [11]:
chatWithDocs("Hi, I would like to do some exercise. I want to gain legs muscles", chat_history)

' You can do exercises like step-ups, single-leg squats, and deadlifts to work your leg muscles. You can also add leg extensions, leg curls, and leg presses if you have a weight bench with attachments. To target more muscle groups in less time, you can add an overhead press at the same time. You can also do forward lunges and twists for more challenge. After a month of exercising, you can reward yourself with a massage.'

In [12]:
chatWithDocs("que es la maquina de turing", chat_history)

" I don't know."

### GET request

In [15]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

class Prompt(BaseModel):
    user_prompt : str

@app.get('/chat_with_docs')
async def Post_prompt(prompt : Prompt):
    return {"response" : chatWithDocs(prompt.user_prompt, chat_history)}

In [16]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://f1e1-181-177-158-4.ngrok.io


INFO:     Started server process [20108]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:64141 - "GET /chat_with_docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:64147 - "GET /chat_with_docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:64150 - "GET /chat_with_docs HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [20108]
